## Kernel to load: vax_inc_general 

In [1]:
import pandas as pd
import numpy as np
import pycountry
import os

In [2]:
notebook_dir = os.path.dirname(os.getcwd())
source_data_path=os.path.join(notebook_dir, "Common Source Data")

In [3]:
df_start= pd.read_csv(os.path.join(source_data_path, "cattle/AllRegions_Cattle.csv"))

#Removing 2024 July-December As this semester is not yet complete to represent true disease incidence
df_start=df_start[df_start['Semester']!='Jul-Dec 2024']
#A few times when country-wide report, they report NaN for administrative division
df_start['Administrative Division'] = df_start['Administrative Division'].fillna(df_start['Country'])
df_start

,Year,Semester,World region,Country,Administrative Division,Disease,Serotype/Subtype/Genotype,Animal Category,Species,Event_id,Outbreak_id,New outbreaks,Susceptible,Measuring units,Cases,Killed and disposed of,Slaughtered,Deaths,Vaccinated
0,2005,Jul-Dec 2005,Africa,Algeria,Algeria,Bovine tuberculosis (-2018),NaN,Domestic,Cattle,-,-,-,1487,Animal,193,0,193,0,-
1,2005,Jul-Dec 2005,Africa,Algeria,Algeria,Brucella abortus (Inf. with),NaN,Domestic,Cattle,-,-,-,2778,Animal,444,0,444,0,0
2,2005,Jul-Dec 2005,Africa,Algeria,Algeria,Echinococcosis/hydatidosis,NaN,Domestic,Cattle,-,-,-,-,Animal,5589,0,5589,0,-
3,2005,Jul-Dec 2005,Africa,Algeria,Algeria,Rabies virus (Inf. with),NaN,Domestic,Cattle,-,-,-,929,Animal,125,46,23,56,-
4,2005,Jul-Dec 2005,Africa,Angola,Bengo,Mycoplasma mycoides subsp. mycoides SC (Inf. w...,NaN,Domestic,Cattle,-,-,-,630,Animal,13,0,0,11,619
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109727,2024,Jan-Jun 2024,Europe,United Kingdom,North Norfolk,Bluetongue virus (Inf. with),3,Domestic,Cattle,5330,-,2,454,Animal,2,0,0,0,0
109728,2024,Jan-Jun 2024,Europe,United Kingdom,South Norfolk,Bluetongue virus (Inf. with),3,Domestic,Cattle,5330,-,13,3517,Animal,28,1,0,0,0
109729,2024,Jan-Jun 2024,Europe,United Kingdom,Swale,Bluetongue virus (Inf. with),3,Domestic,Cattle,5330,132042,1,254,Animal,2,0,0,0,0
109730,2024,Jan-Jun 2024,Europe,United Kingdom,Tandridge,Bluetongue virus (Inf. with),3,Domestic,Cattle,5330,132770,1,690,Animal,1,0,0,0,0


In [4]:
columns_to_process = ['Vaccinated', 'Susceptible', 'Killed and disposed of', 
                      'Deaths', 'New outbreaks', 'Slaughtered', 'Cases']

columns_to_process = ['Vaccinated', 'Susceptible', 'Killed and disposed of', 'Deaths', 'New outbreaks', 'Slaughtered','Cases']
for col in columns_to_process:
    if col=='Cases':
        df_start[col] = df_start[col].replace('-', np.nan).astype(float)    
    else:
        df_start[col] = df_start[col].replace('-', 0).astype(float)

df_start['Total_Removed'] = df_start['Deaths'] + df_start['Killed and disposed of'] + df_start['Slaughtered']

df_start['Adjusted_Susceptible'] = df_start['Susceptible']
df_start['Adjustment_STEP'] = ''  # To record which steps were applied


# Step 1: Add Vaccinated if INITIAL Susceptible < Vaccinated
mask_vaccinated = df_start['Susceptible'] < df_start['Vaccinated']
df_start.loc[mask_vaccinated, 'Adjusted_Susceptible'] +=  df_start.loc[mask_vaccinated, 'Vaccinated']
df_start.loc[mask_vaccinated, 'Adjustment_STEP'] += 'Step1_AddVaccinated;'

# Step 2: Add Total Removed if INITIAL Susceptible < Removed
mask_removed = df_start['Susceptible'] < df_start['Total_Removed']
df_start.loc[mask_removed, 'Adjusted_Susceptible'] += df_start.loc[mask_removed, 'Total_Removed']
df_start.loc[mask_removed, 'Adjustment_STEP'] += 'Step2_AddRemoved;'

# Step 3: If Vaccinated was NOT added but Adjusted Susceptible < Cases and adding Vaccinated alone resolves the discrepancy, add Vaccinated
mask_vaccinated_not_added = (~mask_vaccinated) & \
                            (df_start['Adjusted_Susceptible'] < df_start['Cases']) & \
                            ((df_start['Adjusted_Susceptible'] + df_start['Vaccinated']) >= df_start['Cases'])

df_start.loc[mask_vaccinated_not_added, 'Adjusted_Susceptible'] += df_start.loc[mask_vaccinated_not_added, 'Vaccinated']
df_start.loc[mask_vaccinated_not_added, 'Adjustment_STEP'] += 'Step3_AddVaccinated;'


# Step 4: If Removed was NOT added but Adjusted_Susceptible < Cases and adding Removed alone resolves the discrepancy, add Removed
mask_removed_not_added = (~mask_removed) & \
(df_start['Adjusted_Susceptible'] < df_start['Cases']) &\
(df_start['Adjusted_Susceptible'] + df_start['Total_Removed'] >= df_start['Cases'])
df_start.loc[mask_removed_not_added, 'Adjusted_Susceptible'] += df_start.loc[mask_removed_not_added, 'Total_Removed']
df_start.loc[mask_removed_not_added, 'Adjustment_STEP'] += 'Step5_AddRemoved;'

# Step 5: If Vaccinated and Removed was NOT added but Adjusted_Susceptible < Cases and adding Removed and Vaccinated together resolves the discrepancy
#add Removed and Vaccinted
mask_none_were_added = (~mask_removed) &\
(~mask_vaccinated) & \
(~mask_vaccinated_not_added)&\
(~mask_removed_not_added)&\
(df_start['Adjusted_Susceptible'] < df_start['Cases']) &\
(df_start['Adjusted_Susceptible'] +df_start['Total_Removed'] + df_start['Vaccinated'] >= df_start['Cases'])
df_start.loc[mask_none_were_added, 'Adjusted_Susceptible'] += df_start.loc[mask_none_were_added, 'Total_Removed'] + df_start.loc[mask_none_were_added, 'Vaccinated']
df_start.loc[mask_none_were_added, 'Adjustment_STEP'] += 'Step5_AddRemoved;'

# Step 6: If less than cases after the above steps, just add cases to Adjusted_Susceptible
mask_final_cases =(df_start['Adjusted_Susceptible'] < df_start['Cases'])
df_start.loc[mask_final_cases, 'Adjusted_Susceptible'] += df_start.loc[mask_final_cases,'Cases']
df_start.loc[mask_final_cases, 'Adjustment_STEP'] += 'Step6_AddCases;'

df_start['Final_Adjustment'] = np.where(
    df_start['Adjusted_Susceptible'] != df_start['Susceptible'], 1, 0
)



# Handle invalid Susceptible values
    #administrative divisions with Susceptible = 0 were excluded because vaccination coverage cannot be computed reliably.
df_start = df_start[df_start['Susceptible'] > 0]


unique_countries = df_start['Country'].unique()
print(unique_countries)
print(len(unique_countries))


['Algeria' 'Angola' 'Benin' 'Botswana' 'Burkina Faso' 'Cameroon' 'Chad'
 'Congo (Dem. Rep. of the)' 'Eritrea' 'Eswatini' 'Ethiopia' 'Ghana'
 'Guinea' 'Kenya' 'Lesotho' 'Madagascar' 'Mali' 'Mozambique' 'Namibia'
 'Nigeria' 'Senegal' 'Sudan' 'Tanzania' 'Togo' 'Tunisia' 'Uganda'
 'Zimbabwe' 'Argentina' 'Barbados' 'Brazil' 'Chile' 'Colombia'
 'Costa Rica' 'Cuba' 'Dominican (Rep.)' 'Ecuador' 'El Salvador'
 'Guatemala' 'Nicaragua' 'Panama' 'Peru'
 'Saint Vincent and the Grenadines' 'United States of America' 'Uruguay'
 'Venezuela' 'Afghanistan' 'Armenia' 'Azerbaijan' 'Bhutan'
 'Chinese Taipei' 'Georgia' 'Iran' 'Iraq' 'Israel' 'Kazakhstan'
 'Kyrgyzstan' 'Malaysia' 'Mongolia' 'Myanmar' 'Nepal' 'Pakistan'
 'Saudi Arabia' 'Sri Lanka' 'Thailand' 'Türkiye (Rep. of)' 'Vietnam'
 'Yemen' 'Albania' 'Austria' 'Bulgaria' 'Croatia' 'Czech Republic'
 'Denmark' 'Greece' 'Ireland' 'Italy' 'Latvia' 'Lithuania' 'Luxembourg'
 'Poland' 'Portugal' 'Romania' 'Russia' 'Serbia and Montenegro' 'Slovakia'
 'Spain' 'S

In [5]:
# List of diseases where no vaccine exists
diseases_no_vaccine = [
    "African swine fever virus (Inf. with)",
    "Avian tuberculosis (-2005)",
    "Bovine spongiform encephalopathy",
    "Crimean Congo haemorrhagic fever (2006-)",
    "Maedi-visna",
    "Malignant catarrhal fever (wildebeest only)(2006-2008)",
    "New world screwworm (Cochliomyia hominivorax)",
    "Nipah virus encephalitis",
    "Scrapie",
    "Surra (Trypanosoma evansi)",
    "Theileria equi and Babesia caballi (Inf. with) (Equine piroplasmosis)",
    "Tularemia"
]


In [6]:
records_edited = 0
diseases_found = []
removed_records = []  

# Iterate through diseases and remove invalid records
for disease in diseases_no_vaccine:
    if disease in df_start['Disease'].unique():
        diseases_found.append(disease)  # Track diseases present in the dataframe
        condition = (df_start['Disease'] == disease) & (df_start['Vaccinated'] > 0)
        
        disease_removed_records = df_start[condition]
        removed_records.append(disease_removed_records)
        
        count_disease_edits = disease_removed_records.shape[0]
        records_edited += count_disease_edits
        
        df_start = df_start[~condition]

# Print summary results
print(f"Total records edited (removed): {records_edited}")

if diseases_found:
    print("\nDiseases with removed vaccination records:")
    for d in diseases_found:
        print(f" - {d}")
else:
    print("\nNone of the specified diseases were found in the dataframe.")

if removed_records:
    print("\nDetails of removed records:")
    for i, records in enumerate(removed_records):
        print(f"\nRemoved records for disease: {diseases_found[i]}")
        print(records)
else:
    print("\nNo records were removed.")

df_start.reset_index(drop=True, inplace=True)
print("\nFinal dataframe shape:", df_start.shape)


Total records edited (removed): 0

Diseases with removed vaccination records:
 - African swine fever virus (Inf. with)
 - Bovine spongiform encephalopathy
 - Crimean Congo haemorrhagic fever (2006-)
 - Maedi-visna
 - Malignant catarrhal fever (wildebeest only)(2006-2008)
 - New world screwworm (Cochliomyia hominivorax)
 - Scrapie
 - Surra (Trypanosoma evansi)
 - Theileria equi and Babesia caballi (Inf. with) (Equine piroplasmosis)
 - Tularemia

Details of removed records:

Removed records for disease: African swine fever virus (Inf. with)
Empty DataFrame
Columns: [Year, Semester, World region, Country, Administrative Division, Disease, Serotype/Subtype/Genotype, Animal Category, Species, Event_id, Outbreak_id, New outbreaks, Susceptible, Measuring units, Cases, Killed and disposed of, Slaughtered, Deaths, Vaccinated, Total_Removed, Adjusted_Susceptible, Adjustment_STEP, Final_Adjustment]
Index: []

[0 rows x 23 columns]

Removed records for disease: Bovine spongiform encephalopathy
Emp

In [7]:
dict_dates = dict({
                  'Jan-Jun 2005':'2005-06-30','Jul-Dec 2005':'2005-12-31',
                  'Jan-Jun 2006':'2006-06-30','Jul-Dec 2006':'2006-12-31',
                  'Jan-Jun 2007':'2007-06-30','Jul-Dec 2007':'2007-12-31',
                  'Jan-Jun 2008':'2008-06-30','Jul-Dec 2008':'2008-12-31',
                  'Jan-Jun 2009':'2009-06-30','Jul-Dec 2009':'2009-12-31',
                  'Jan-Jun 2010':'2010-06-30','Jul-Dec 2010':'2010-12-31',
                  'Jan-Jun 2011':'2011-06-30','Jul-Dec 2011':'2011-12-31',
                  'Jan-Jun 2012':'2012-06-30','Jul-Dec 2012':'2012-12-31',
                  'Jan-Jun 2013':'2013-06-30','Jul-Dec 2013':'2013-12-31',
                  'Jan-Jun 2014':'2014-06-30','Jul-Dec 2014':'2014-12-31',
                  'Jan-Jun 2015':'2015-06-30','Jul-Dec 2015':'2015-12-31',
                  'Jan-Jun 2016':'2016-06-30','Jul-Dec 2016':'2016-12-31',
                  'Jan-Jun 2017':'2017-06-30','Jul-Dec 2017':'2017-12-31',
                  'Jan-Jun 2018':'2018-06-30','Jul-Dec 2018':'2018-12-31',
                  'Jan-Jun 2019':'2019-06-30','Jul-Dec 2019':'2019-12-31',
                  'Jan-Jun 2020':'2020-06-30','Jul-Dec 2020':'2020-12-31',
                  'Jan-Jun 2021':'2021-06-30','Jul-Dec 2021':'2021-12-31',
                  'Jan-Jun 2022':'2022-06-30','Jul-Dec 2022':'2022-12-31',
                  'Jan-Jun 2023':'2023-06-30','Jul-Dec 2023':'2023-12-31',
                  'Jan-Jun 2024':'2024-06-30'

                                                                        })

df_start['time']=[dict_dates[i] for i in df_start['Semester']]
df_start['Semester']=['June' if '06-30' in time else 'December' for time in df_start['time'].values]
df_start['time'] = pd.to_datetime(df_start['time'])

In [8]:
start_countries=df_start['Country']

countries = {}
for country in pycountry.countries:
    countries[country.name] = country.alpha_3
    countries['USA']='USA'
    countries['UK']='GBR'
    countries['Taiwan']='TWN'
    countries['South Korea']='KOR'
    countries['Czech Republic']='CZE'
    countries['Brunei']='BRN'
    countries['Russia']='RUS'
    countries['Iran']='IRN'
    countries['United States of America']='USA'
    countries['Venezuela']='VEN'
    countries['China (Hong Kong SAR)']='HKG'
    countries["Cote d'Ivoire"]='CIV'
    countries['DR Congo']='COD'
    countries['Guinea Bissau']='GNB'
    countries['Lao PDR']='LAO'
    countries['Micronesia (Federated States of)']='FSM'
    countries['North Korea']='PRK'
    countries['Occupied Palestinian Territory']='PSE'
    countries['Swaziland']='SWZ'
    countries['Tanzania']='TZA'
    countries['Bolivia']='BOL'
    countries['Macedonia (TFYR)']='MKD'
    countries['Moldova']='MDA'
    countries['Bolivia (Plurinational State of)']='BOL'
    countries['China, Hong Kong SAR']='HKG'
    countries['China, Taiwan Province of']='TWN'
    countries['China, mainland']='CHN'
    countries['Czechoslovakia']='CSK'
    countries["Democratic People's Republic of Korea"]='PRK'
    countries['Democratic Republic of the Congo']='COD'
    countries['French Guyana']='GUF'
    countries['Micronesia']='FSM'
    countries['Palestine']='PSE'
    countries['Polynesia']='PYF'
    countries['Republic of Korea']='KOR'
    countries['Serbia and Montenegro']='SCG'
    countries['Sudan (former)']='SDN'
    countries['Türkiye']='TUR'
    countries['USSR']='SUN'
    countries['Iran (Islamic Republic of)']='IRN'
    countries['Republic of Moldova']='MDA'
    countries['United Kingdom of Great Britain and Northern Ireland']='GBR'
    countries['United Republic of Tanzania']='TZA'
    countries['Venezuela (Bolivarian Republic of)']='VEN'
    countries['Yugoslav SFR']='YUG'
    countries['Ethiopia PDR']='ETH'
    countries['Central African (Rep.)']='CAF'
    countries["China (People's Rep. of)"]='CHN'
    countries['Chinese Taipei']='TWN'
    countries['Congo (Dem. Rep. of the)']='COD'
    countries['Congo (Rep. of the)']='COG'
    countries["Cote D'Ivoire"]='CIV'
    countries['Dominican (Rep.)']='DOM'
    countries["Korea (Dem People's Rep. of)"]='PRK'
    countries['Korea (Rep. of)']='KOR'
    countries['Laos']='LAO'
    countries['South Sudan (Rep. of)']='SSD'
    countries['Syria']='SYR'
    countries['St. Vincent and the Grenadines']='VCT'
    countries['Vietnam']='VNM'
    countries['Reunion']='REU'
    countries['Guadaloupe']='GLP'
    countries['China, Macao SAR']='MAC'
    countries['Netherlands (Kingdom of the)']='NLD'
    countries['Türkiye (Rep. of)']='TUR'
    countries['Belgium-Luxembourg']='BLX'
    countries['St. Helena']='SHN'
    countries['Cabo verde']='CPB'
    countries['Faeroe Islands']='FRO'
    countries['Wallis and Futuna Islands']='WLF'


codes_start=[countries.get(country, 'Unknown code:'+country) for country in start_countries]

for code in codes_start:
    if "Unknown" in code:
        print("FIX THIS:",code)


iso3s_start=[]

for i in start_countries:
    try:
        iso3s_start+=[countries[i]]
    except:
        iso3s_start+=[None]
        
df_start['ISO3']=iso3s_start

In [9]:
df_start = (
    df_start.groupby(['Country', 'Administrative Division', 'Disease', 'Semester', 'Year'], as_index=False)
    .agg({
        'Country': 'first',
        'ISO3': 'first',
        'Administrative Division': 'first',
        'Disease': 'first',
        'Semester': 'first',
        'Year': 'first',
        'Adjusted_Susceptible': 'sum',
        'Vaccinated': 'sum',
        'World region': 'first',  
        'Serotype/Subtype/Genotype':lambda x: 'aggregated',
        'Event_id':lambda x: 'aggregated',
        'Species':'first',
        'time':'first',
        'Outbreak_id':lambda x: 'aggregated',
        'New outbreaks':'sum'
    })
)

df_start['original_year']=df_start['Year']

In [10]:
# Add missing years for each unique combination of Semester, ISO3, Administrative Division, and Disease
def add_missing_years(df):
    group_columns = ["Semester", "ISO3", "Administrative Division", "Disease"]

    unique_combinations = df[group_columns].drop_duplicates()
    unique_combinations['Min_Year'] = df.groupby(group_columns)['Year'].transform('min')
    unique_combinations['Max_Year'] = df.groupby(group_columns)['Year'].transform('max')

    all_years = []
    for _, row in unique_combinations.iterrows():
        years = pd.DataFrame({'Year': range(row['Min_Year'], row['Max_Year'] + 1)})
        for col in group_columns:
            years[col] = row[col]
        all_years.append(years)

    all_years_df = pd.concat(all_years, ignore_index=True)

    expanded_df = pd.merge(all_years_df, df, on=group_columns + ['Year'], how='left')

    none_columns = ["Vaccinated", 'Adjusted_Susceptible']
    for col in none_columns:
        expanded_df[col] = expanded_df[col].where(expanded_df[col].notna(), None)

    ffill_columns = expanded_df.columns.difference(none_columns + ['Year'])

    expanded_df[ffill_columns] = expanded_df.sort_values(by=group_columns + ['Year'])[ffill_columns].ffill()

    def update_time(row):
        if row['Semester'] == 'December':
            return f"{row['Year']}-12-31"
        elif row['Semester'] == 'June':
            return f"{row['Year']}-06-30"
        else:
            return None  

    expanded_df['time'] = expanded_df.apply(update_time, axis=1)

    expanded_df['time'] = pd.to_datetime(expanded_df['time'], format='%Y-%m-%d', errors='coerce')

    expanded_df['Derived_Vaccinated_Method'] = "None"

    return expanded_df


    expanded_df['time'] = expanded_df.apply(update_time, axis=1)

    expanded_df['time'] = pd.to_datetime(expanded_df['time'], format='%Y-%m-%d', errors='coerce')

    expanded_df['Derived_Vaccinated_Method'] = "None"

    return expanded_df

    def update_time(row):
        if row['Semester'] == 'December':
            return f"{row['Year']}-12-31"
        elif row['Semester'] == 'June':
            return f"{row['Year']}-06-30"
        else:
            return None  

    expanded_df['time'] = expanded_df.apply(update_time, axis=1)

    expanded_df['time'] = pd.to_datetime(expanded_df['time'], format='%Y-%m-%d', errors='coerce')

    expanded_df['Derived_Vaccinated_Method'] = "None"

    return expanded_df


# Interpolate Adjusted_Susceptible and Vaccinated
def interpolate_adjusted_and_cases(df):
    group_columns = ["Semester", "ISO3", "Administrative Division", "Disease"]

    def interpolate_group(group):
        group = group.sort_values('Year').reset_index(drop=True)

        interpolated_susceptible = group['Adjusted_Susceptible'].fillna(value=np.nan).interpolate(method='linear')
        interpolated_vaccinated = group['Vaccinated'].fillna(value=np.nan).interpolate(method='linear')

        group.loc[interpolated_susceptible.notna() & group['Adjusted_Susceptible'].isna(), 'Derived_Vaccinated_Method'] = "Adjusted_Susceptible_Vaccinated"
        group.loc[interpolated_vaccinated.notna() & group['Vaccinated'].isna(), 'Derived_Vaccinated_Method'] = "Adjusted_Susceptible_Vaccinated"

        group['Adjusted_Susceptible'] = interpolated_susceptible
        group['Vaccinated'] = interpolated_vaccinated

        return group

    return df.groupby(group_columns, group_keys=False).apply(interpolate_group)



# Function to to collect Source year information
def update_interpolated_upper_year(df):
    group_columns = ["Semester", "ISO3", "Administrative Division", "Disease"]

    def assign_upper_year(group):
        group = group.sort_values('Year').reset_index(drop=True)

        for idx in group[group['Derived_Vaccinated_Method'].notna()].index:
            if group.loc[idx, 'Derived_Vaccinated_Method'] in ["Adjusted_Susceptible_Vaccinated"]:
                upper_idx = group[(group.index > idx) & (group['Derived_Vaccinated_Method']=='None')].index.min()
                lower_idx = group[(group.index < idx) & (group['Derived_Vaccinated_Method']=='None')].index.max()

                
                if pd.notna(upper_idx): 
                    group.loc[idx, 'interpolated_upper_year'] = int(group.loc[upper_idx, 'Year'])
                else:
                    group.loc[idx, 'interpolated_upper_year'] = None  # Handle edge cases

                if pd.notna(lower_idx): 
                    group.loc[idx, 'interpolated_lower_year'] = int(group.loc[lower_idx, 'Year'])
                else:
                    group.loc[idx, 'interpolated_lower_year'] = None  # Handle edge cases
                
        return group

    return df.groupby(group_columns, group_keys=False).apply(assign_upper_year)


def process_dataframe(df):
    df = add_missing_years(df)  # Add missing years
    df = interpolate_adjusted_and_cases(df)  # Interpolate Adjusted_Susceptible and Incidence, update Cases
    df=update_interpolated_upper_year(df)
    return df

df_result = process_dataframe(df_start)

In [11]:
final_dfs=[]

for year in range(2005,2025): 
    
    df2=df_result.copy()
    df2=df2[df2['Year']<=year]
    
    #Decided to pool across both semesters of the year to get a more accurate 'year' vaccination coverage, if it's affected by temporal trends
    df2 = df2[df2['time'] == df2.groupby(['Country', 'Administrative Division', 'Disease','Semester'])['time'].transform('max')]
    cols_keep=['Country','ISO3','Administrative Division','Disease','Semester','Year','Adjusted_Susceptible','Vaccinated','World region','Serotype/Subtype/Genotype','Event_id','Species','time','Outbreak_id','New outbreaks','original_year','Derived_Vaccinated_Method','interpolated_upper_year',
              'interpolated_lower_year']
    df2=df2.loc[:,cols_keep]

    df2 = df2.sort_values('time')


    year_range = {}
    
    # Populate year_range with valid min and max years
    for row in df2.itertuples(index=False):
        key = (row.Country, row.Disease)
        if key not in year_range:
            year_range[key] = [9999, 0]  # Default range
    
        # Update with interpolated_upper_year
        if not pd.isna(row.interpolated_upper_year):
            year_range[key][1] = max(year_range[key][1], int(row.interpolated_upper_year))
        # Update with interpolated_lower_year
        if not pd.isna(row.interpolated_lower_year):
            year_range[key][0] = min(year_range[key][0], int(row.interpolated_lower_year))
        # Update with original_year
        if not pd.isna(row.original_year):
            year_range[key][0] = min(year_range[key][0], int(row.original_year))
            year_range[key][1] = max(year_range[key][1], int(row.original_year))
    
    # Format year_range for consistent handling
    for key in year_range:
        start, end = year_range[key]
        if start == 9999 and end == 0:  # No valid years
            year_range[key] = [9999, 0]
        elif start == end:  # Single year
            year_range[key] = [start,0]
        else:  # Valid range
            year_range[key] = [start, end]
       
    
    df_final = (
        df2.groupby(['Country', 'Disease'], as_index=False)
        .agg({
            'Adjusted_Susceptible': 'sum',
            'Vaccinated': 'sum',
            'New outbreaks': 'sum',
    
            'ISO3': 'first',  
            'original_year': 'first',
            'Year': 'first',
            'Derived_Vaccinated_Method': lambda x: 'None' if all(v == 'None' for v in x) else x[x != 'None'].iloc[0]
        })
        .drop(columns=['Year'])  
    )    
    
    df_final['Vaccine Coverage']=df_final['Vaccinated']/(df_final['Adjusted_Susceptible'])
    df_final=df_final.reset_index()
    
    df_final['Year']=[year]*len(df_final)
    
    year_range_col=[]
    for row in df_final.iterrows():
        if year_range[(row[1]['Country'],row[1]['Disease'])]!=[9999,0]:
            if year_range[(row[1]['Country'],row[1]['Disease'])][0]==9999:
                year_range_col+=[str(year_range[(row[1]['Country'],row[1]['Disease'])][1])]
            elif year_range[(row[1]['Country'],row[1]['Disease'])][1]==0:
                year_range_col+=[str(year_range[(row[1]['Country'],row[1]['Disease'])][0])]
            else:
                year_range_col+=[str(year_range[(row[1]['Country'],row[1]['Disease'])][0])+'-'+str(year_range[(row[1]['Country'],row[1]['Disease'])][1])]
    df_final['Year Range']=year_range_col        
    final_dfs+=[df_final]
    
    print("Finished analysis for:",str(year))

Finished analysis for: 2005
Finished analysis for: 2006
Finished analysis for: 2007
Finished analysis for: 2008
Finished analysis for: 2009
Finished analysis for: 2010
Finished analysis for: 2011
Finished analysis for: 2012
Finished analysis for: 2013
Finished analysis for: 2014
Finished analysis for: 2015
Finished analysis for: 2016
Finished analysis for: 2017
Finished analysis for: 2018
Finished analysis for: 2019
Finished analysis for: 2020
Finished analysis for: 2021
Finished analysis for: 2022
Finished analysis for: 2023
Finished analysis for: 2024


In [12]:
years_data=pd.concat(final_dfs)
years_data['Disease']=[i if i!= 'Newcastle disease virus (Inf. with)' else 'Newcastle disease (velogenic)' for i in years_data['Disease']]
years_data.sort_values(['ISO3','Year','Disease'])
years_data.to_csv('2005-2024_part_cattle_vaccine_coverage_by_country.csv',index=False)